In [3]:
#Environment check /installs
import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())

Python: 3.13.7 (tags/v3.13.7:bcee1c3, Aug 14 2025, 14:15:11) [MSC v.1944 64 bit (AMD64)]
Platform: Windows-11-10.0.26200-SP0


In [4]:
#Imports & paths (annotated)
import pandas as pd             # Core data handling: read, clean, and transform CSVs
import numpy as np              # Numerical operations, NaN handling, and unit conversions
import re                       # Regular expressions for parsing messy strings (coords, units)
from pathlib import Path         # Cross-platform file path handling for locating input/output files
from datetime import datetime    # Working with timestamps and generating sample date ranges
from dateutil import parser as dateparser  # Flexible parsing of inconsistent date formats

RAW_DIR = Path("../data")   #location of csvs
PATH_STATIONS = RAW_DIR / "raw_station_metadata.csv"
PATH_SAMPLES = RAW_DIR / "raw_water_samples.csv"

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 140)

In [5]:
#Inspect Raw Structure (diagnose junk headers, footers, columns)

#Read just a small sample from the start and end of each CSV to preview structure
def quick_peek_csv(path, n=5): #5 get used by default unless specified, ie. quick_peek_csv(PATH_STATIONS, n=10)
    """Peek at the first and last few lines of a CSV file to detect header/footer artifacts."""
    print(f"\n==={path.name} ===")
    with open(path, "r", encoding="utf-8", errors="ignore") as f:   #open file located at path, in read mode ("r"), utf-8 ensures special characters read correctly, skips errors instead of failing to execute code
        lines = f.readlines()   #reads all lines
    print("\n--- FIRST FEW LINES ---")
    for l in lines[:n]: #limits line read to first 5 (or specified)
        print(l.strip())
    print("\n--- LAST FEW LINES ---")
    for l in lines[-n:]:    #limits line read to last 5 (or specified) note colon placement different
        print(l.strip())

quick_peek_csv(PATH_STATIONS)
quick_peek_csv(PATH_SAMPLES)

#PICK UP HERE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


===raw_station_metadata.csv ===

--- FIRST FEW LINES ---
THIS IS A HEADER,foo, Station ID ,SiteName,Latitude,Longitude,Coordinates,Elev,Agency_Code,CRS,Notes
StationMetadata v0.1,bar,,,,,,,,,
,,S001,River Site 1,37.872701,-78.532764,,246.9 m,City Lab,,
,,S002,River Site 2,40.753572,-77.566062,,76.5 m,EPA,,Near bridge
,,S003,River Site 3,,,"39.65997,-75.801949",35.4 m,USGS,EPSG:4326,Upstream

--- LAST FEW LINES ---
,,S012,River Site 12,40.849549,-78.40261,,203.1 m,EPA,,Downstream
,,S013,River Site 13  ,"40°9'43""N","75°53'10""S",,23.4 m,City Lab,EPSG:4269,
,,S014,River Site 14,"37°3'42""N","75°15'38""S",,364.6 m,DWR,WGS84,
,,S015,River Site 15,36.909125,-79.628397,,110.9 m,USGS,EPSG:4326,Near bridge
,,S004,River Site 4,"38°59'35""N","76°32'39""S",,380.1 m,USGS,,Downstream

===raw_water_samples.csv ===

--- FIRST FEW LINES ---
Sample ID,StationCode,Sample Date,Analyte,Result,Units,Method ID,DetectLimit,Temp,Remarks,footer
WQ1000,S014,2025-10-20,Chloride,2698.40,ug/L,SM 4500-NO3,0.07,66.

In [6]:
# Load minimally to inspect column names (no cleaning yet)
stations_probe = pd.read_csv(PATH_STATIONS, nrows=10, dtype=str)
samples_probe  = pd.read_csv(PATH_SAMPLES,  nrows=10, dtype=str)

print("\n--- Column names: raw_station_metadata.csv ---")
print(stations_probe.columns.tolist())

print("\n--- Column names: raw_water_samples.csv ---")
print(samples_probe.columns.tolist())


--- Column names: raw_station_metadata.csv ---
['THIS IS A HEADER', 'foo', ' Station ID ', 'SiteName', 'Latitude', 'Longitude', 'Coordinates', 'Elev', 'Agency_Code', 'CRS', 'Notes']

--- Column names: raw_water_samples.csv ---
['Sample ID', 'StationCode', 'Sample Date', 'Analyte', 'Result', 'Units', 'Method ID', 'DetectLimit', 'Temp', 'Remarks', 'footer']


In [7]:
#Robust Load (handle junk header/footer rows, everything as string)

def read_raw(path):
    df = pd.read_csv(path, dtype=str, keep_default_na=False, na_values=["", " "])   #dont translate the default conditions as NA (b/c some might be legitamit data), just "" and " "
    #Drop obvious junk rows
    #df = df[[c for c in df.columns if not c.startswith("Unnamed")]]  **left this out b/c practice dataset doesnt include "Unnamed" in any cell
    df = df[~df.apply(lambda r: r.astype(str).str.contains("End of File|StationMetadata v0.1", case=False).any(), axis=1)]  #.apply(..axis=1) means apply function to every row, lambda r is an anonymous function operating on each row "r", case=False means ignore casing, .any() returns True if any column in row matched pattern
    return df

raw_stations = read_raw(PATH_STATIONS)
raw_samples = read_raw(PATH_SAMPLES)

raw_stations.head(3), raw_samples.head(3)

(  THIS IS A HEADER  foo  Station ID       SiteName   Latitude   Longitude          Coordinates     Elev Agency_Code        CRS  \
 1              NaN  NaN         S001  River Site 1  37.872701  -78.532764                  NaN  246.9 m    City Lab        NaN   
 2              NaN  NaN         S002  River Site 2  40.753572  -77.566062                  NaN   76.5 m         EPA        NaN   
 3              NaN  NaN         S003  River Site 3        NaN         NaN  39.65997,-75.801949   35.4 m        USGS  EPSG:4326   
 
          Notes  
 1               
 2  Near bridge  
 3     Upstream  ,
   Sample ID StationCode  Sample Date   Analyte   Result Units    Method ID DetectLimit    Temp Remarks footer
 0    WQ1000        S014   2025-10-20  Chloride  2698.40  ug/L  SM 4500-NO3        0.07  66.7 F     NaN    NaN
 1    WQ1001        S009  25-Oct-2025  AMMONIUM       ND  mg/L  SM 4500-NH3        0.11  60.8 F     NaN    NaN
 2    WQ1002        S006   2025-10-17  Ammonium    <0.68  mg/L     I

In [8]:
#Standardize column names (snake_case, trim)

#Column name normalization
def to_snake(s):
    #regular expression (regex) string cleaning.  s is variable holding any string
    s = re.sub(r"\s+", "_", s.strip())  #re=regular expression function, sub() substitute, r"\s+" \s any whitespace char, + one or more in a row, repalce with _
    s=re.sub(r"[^\w]+", "_", s) #\w any word char, ^ not: [^\w]+ one or more non-word char's(not letter digit or underscore): includes punctuation, symbols, special chars, replace with _
    return s.lower().strip("_")

raw_stations.columns = [to_snake(c) for c in raw_stations.columns]
raw_samples.columns = [to_snake(c) for c in raw_samples.columns]

#Trim all string cells
def trim_df(df):
    return df.map(lambda x: x.strip() if isinstance(x, str) else x) #older ver. pandas is applymap()

raw_stations = trim_df(raw_stations)
raw_samples = trim_df(raw_samples)

raw_stations.columns, raw_samples.columns

(Index(['this_is_a_header', 'foo', 'station_id', 'sitename', 'latitude', 'longitude', 'coordinates', 'elev', 'agency_code', 'crs', 'notes'], dtype='object'),
 Index(['sample_id', 'stationcode', 'sample_date', 'analyte', 'result', 'units', 'method_id', 'detectlimit', 'temp', 'remarks', 'footer'], dtype='object'))

In [ ]:
#Clean and Standardize STATIONS: LATITUDE AND LONGITUDE

#Stations:pick canonical Keys/columns
stations = raw_stations.copy()

#Normalize whitespace and remove accidental trailing spaces in IDs
stations["station_id"] = stations["station_id"].str.strip()

#If coordinates sometimes appear as "lat,lon" in single field, split them (we know "coordinates" column exists, see above printouts)
if "coordinates" in stations.columns:
    coords_split = stations["coordinates"].str.split(",", n=1, expand=True)
    if isinstance(coords_split, pd.DataFrame) and coords_split.shape[1] == 2:   #code only proceeds if the above resulted in a DataFrame(structural check) and a pair of data, rather than a single or no data in a row coords_split.shape[1] == 2
        stations["lat_from_coords"] = coords_split[0].str.strip()
        stations["lon_from_coords"] = coords_split[1].str.strip()

#Prefer explicity latitude/longitude columns if they look valid, else fallback to ..._from_coords
lat_col = "latitude" if "latitude" in stations.columns else "lat_from_coords"
lon_col = "longitude" if "longitude" in stations.columns else "lon_from_coords"

# Parse decimal or DMS strings to decimal degrees
def parse_coord(val):
    if pd.isna(val) or val == "":
        return np.nan
    s = str(val).strip()
    # is value a decimal?
    try:
        return float(s)
    except ValueError:
        pass
    # is value DMS? like 40°26'46"N
    m = re.match(r"^\s*(\d+)[^\d]+(\d+)[^\d]+(\d+)\s*([NSEW])\s*$", s) 
    #r means raw string, tells python treat \ literally.  ^ beginning of string. $ end of string.  \s whitespace, * zero or more: \s* means allow some whitespace
    #\d any digit, + one or more digist, () capture this portion as a group ie. (\d+).  [^\d]+ means "at the start" one or more characters that are not digits(like punctuation/symbols).
    #([NSEW]) captures one uppercase character from the set N, S, E, or W.
    # + between bracketed groups means "join", its just bridge between definitions of the regular expressions
    if m:
        d, mi, sec, hemi = m.groups()
        dec = float(d) + float(mi)/60 + float(sec)/3600
        if hemi in ["S", "W"]:
            dec = -dec
        return dec
    # If "lon,lat" sneaks in
    if "," in s:
        try:
            a, b = s.split(",", 1)
            # we won't assign here; we parse elsewhere
            return np.nan
        except Exception:
            return np.nan
    return np.nan

#Puts our final latitude and longitude data in columns "lat" and "lon" after all the above checks
stations["lat"] = stations.get(lat_col, "").apply(parse_coord)
stations["lon"] = stations.get(lon_col, "").apply(parse_coord)

#If lat/lon were swapped in some rows (e.g., lat looks like -75) try swapping when it makes sense
def maybe_swap(row):
    lat, lon = row["lat"], row["lon"]
    if pd.notna(lat) and pd.notna(lon):
        #Lat should be roughly -90..90; lon -180..180
        if (lat < -90 or lat > 90) and (-90 <= lon <= 90):  #checks if lat seem too large and lon seems small, condition to swap them
            return pd.Series({"lat": lon, "lon": lat})
    return pd.Series({"lat": lat, "lon": lon})  #creates since row (or column) of data as one-dimentional array

stations[["lat", "lon"]] = stations.apply(maybe_swap, axis=1) #double brackets says "select these two columns" in DF "stations"




In [ ]:
#Clean and Standardize STATIONS: ELEVATION, AGENCY CODES, MISC

#Elevation: convert to meters (if ft present)
def parse_elev(s):
    if pd.isna(s) or s == "":
        return np.nan
    m = re.match(r"^\s*([-+]?\d+(\.\d+)?)\s*(ft|feet|foot|m|meters?)?\s*$", s, flags=re.I) #means?

#PICK UP HERE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [11]:
# Cell 8 — Optional: PostGIS connection imports (annotated)

from sqlalchemy import create_engine, text  
# → SQLAlchemy is the core interface to databases.
#   `create_engine()` builds a connection string for PostgreSQL/PostGIS.
#   `text()` allows you to execute raw SQL commands (e.g., creating schemas, geometry columns).

import psycopg2  
# → PostgreSQL driver used under the hood by SQLAlchemy to communicate with the PostGIS container.
#   Required for executing SQL and writing DataFrames to PostGIS tables.

# (Optional, for geospatial validation later)
# from shapely.geometry import Point  
# → If you later want to create geometry objects or verify coordinates before sending to PostGIS.

# from geopandas import GeoDataFrame  
# → Optional upgrade of pandas DataFrames into GeoDataFrames for spatial operations before export.
